In [5]:
from dotenv import load_dotenv
load_dotenv() 

True

In [7]:
import os

claude_key = os.getenv("CLAUDE_KEY")

In [8]:
import anthropic

In [9]:
from sklearn.model_selection import KFold
from datasets import load_dataset, DatasetDict, Dataset, concatenate_datasets
import datasets
import pandas as pd
import os
import logging
import nltk
import numpy as np
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from random import sample


validate = datasets.load_from_disk("./QTSumm/validate")
validate

/blue/cai6307/y.khan/conda/envs/hf-llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FileNotFoundError: Directory ./QTSumm/validate not found

In [ ]:
import pandas as pd

def to_pandas(item):
  return pd.DataFrame(item['table']["rows"],columns=item['table']["header"])

In [ ]:
# Set up the client
client = anthropic.Client(api_key = claude_key)

In [5]:
def create_prompt(table, question, title):
    table_str = to_pandas(item).to_markdown()
    prompt = f"You are an AI assistant tasked with summarizing key points from tabular data based on a given query. The title of the table is {title}. \n\nTable data: \n\n{table_str}. \n\nQuery: \n\n{question}\n\nProvide a concise summary paragraph that captures the main points from the tabular data relevant to the given query." 
    return prompt

In [6]:
from anthropic import HUMAN_PROMPT, AI_PROMPT
from IPython.display import Markdown, display

summaries = []

def generate_summary(item):
    question = item['query']
    title = item['table']["title"]
    table = to_pandas(item)
    prompt = create_prompt(table, question, title)

    # Call the completions API
    completion = client.completions.create(
            model="claude-v1",
            prompt = f"{HUMAN_PROMPT} {prompt} {AI_PROMPT}",
            max_tokens_to_sample=500,
    )

    # Return the summary
    return Markdown(completion.completion)

In [7]:
import time
count = 1
for item in validate:
    print(count)
    count += 1
    summary = generate_summary(item)
    summaries.append(summary)
    time.sleep(30)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200


In [8]:
summaries[4]

 Despite ranking in the bottom half of the Big Ten conference in 1919, a few teams had notable statistics. Minnesota finished with a 4-2-1 overall record and 3-2 conference record, ranking fourth in the conference. They scored 18.6 points per game while only allowing 6.4 points against per game, good for second and third in the conference respectively. Indiana and Purdue also had respectable overall records at 3-4 and 2-4-1, respectively, despite their lower rankings and struggling in conference play. Overall, some of the lower ranked teams in the 1919 Big Ten conference were able to achieve success and notable statistics outside of their conference performance and rankings.

In [10]:
from IPython.display import Markdown

# Convert Markdown objects to strings
markdown_strings = [obj.data for obj in summaries]

validate = validate.add_column("generated_summary", markdown_strings)
validate.save_to_disk("./QTSumm/claude-v1")

Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 19830.29 examples/s]


In [13]:
validate[10]['summary']

'Vinu Mohan get Asianet award for Best New person for act as Mohan Krishnan in film Nivedyam. This movie come out in year 2007. It his second movie, after he first do in Kanne Madanguka in 2005.'

In [14]:
validate[10]['generated_summary']

' Vinu Mohan won the Asianet award for Best Newcomer for his role as Mohan Krishnan in the 2007 film Nivedyam.'

In [8]:
import datasets
validate = datasets.load_from_disk("/home/y.khan/cai6307-y.khan/Query-Focused-Tabular-Summarization/data/claude/claude-v1")

In [17]:
example = validate.filter(lambda x: x["example_id"] == "1ae05e92-4ca6-483d-9307-488d28b0b31b")

Filter: 100%|██████████| 200/200 [00:00<00:00, 1704.77 examples/s]


In [20]:
suma = example['generated_summary']

In [22]:
print(suma[0])

 The players who attended Brigham Young University (Byu) and played for the Utah Jazz are:

Andy Toolson, number 5, played as a guard and forward for the Jazz from 1990 to 1991 and again from 1995 to 1996.


In [5]:
import evaluate
rouge = evaluate.load('rouge')
bertscore = evaluate.load("bertscore")

pred = validate['generated_summary']
ref = validate['summary']
sc = rouge.compute(predictions=pred, references=ref)
bert_score = bertscore.compute(predictions=pred, references=ref, lang = "en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
import numpy as np
sc['rougeLsum'], np.mean(bert_score['f1'])

(0.3711429403296882, 0.9011911398172379)

In [15]:
import evaluate

rouge = evaluate.load('rouge')
rouge_results = rouge.compute(predictions=validate['generated_summary'], references=validate['summary'])
print(rouge_results)

2024-03-26 23:21:35.454839: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 23:21:37.195324: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


{'rouge1': 0.4916664448292566, 'rouge2': 0.24649439518691946, 'rougeL': 0.3688335814276131, 'rougeLsum': 0.37027148460221343}


In [33]:
bertscore = evaluate.load("bertscore")
bert_score = bertscore.compute(predictions=validate['generated_summary'], references=validate['summary'], lang = "en")

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [34]:
import statistics

precision = statistics.mean(bert_score['precision'])
recall = statistics.mean(bert_score['recall'])
f1 = statistics.mean(bert_score['f1'])
print("Precision:", precision)
print("\nRecall:", recall)
print("\nf1:", f1) 

Precision: 0.895864452123642

Recall: 0.907031309902668

f1: 0.9011911398172379


In [35]:
bleu = evaluate.load("bleu")
bleu_result = bleu.compute(predictions=validate['generated_summary'], references=validate['summary'])
print(bleu_result)

{'bleu': 0.14202644137088632, 'precisions': [0.4123116563042626, 0.18172548575312977, 0.0968887870280181, 0.05604822349736278], 'brevity_penalty': 1.0, 'length_ratio': 1.328644786782764, 'translation_length': 17853, 'reference_length': 13437}


In [31]:
bert_score.keys()

dict_keys(['precision', 'recall', 'f1', 'hashcode'])

In [17]:
from rouge import Rouge

ref = validate[4]['summary']
pred = completion.completion
rouge = Rouge()

rouge_results = rouge.get_scores(pred, ref)
print(rouge_results)

NameError: name 'completion' is not defined

In [10]:
rouge_scores_out = []
for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = rouge_results[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score
        }
        rouge_scores_out.append(row)

In [11]:
rouge_scores_out

[{'Metric': 'rouge-1 (F-Score)', 'Summary 1': 0.7391304298416824},
 {'Metric': 'rouge-2 (F-Score)', 'Summary 1': 0.33846153353727815},
 {'Metric': 'rouge-l (F-Score)', 'Summary 1': 0.7173912994068998}]

In [12]:
# BERTScore leverages the pre-trained contextual embeddings from BERT and matches words in candidate and reference sentences by cosine similarity.
from bert_score import BERTScorer

# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

# Calculate BERTScore for the summary 1 against the excerpt
# P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
P1, R1, F1_1 = scorer.score([pred], [ref])

print("Summary 1 F1 Score:", F1_1.tolist()[0])

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary 1 F1 Score: 0.9167320132255554


In [13]:
print(prompt)

You are an AI assistant tasked with summarizing key points from tabular data based on a given query. The title of the table is 2008–09 Stanford Cardinal women's basketball team - Schedule. 

Table data: 

|    | Date       | Location      | Opponent         |   Cardinal Points |   Opp. Points | Record   |
|---:|:-----------|:--------------|:-----------------|------------------:|--------------:|:---------|
|  0 | Nov. 14/08 | Stanford      | Minnesota        |                68 |            55 | 1-0      |
|  1 | Nov. 16/08 | Waco, TX      | Baylor           |                65 |            81 | 1-1      |
|  2 | Nov. 20/08 | Stanford      | New Mexico       |                84 |            46 | 2-1      |
|  3 | Nov. 23/08 | Stanford      | Rutgers          |                81 |            47 | 3-1      |
|  4 | Nov. 28/08 | Honolulu      | Purdue           |                78 |            70 | 4-1      |
|  5 | Nov. 29/08 | Honolulu      | Iowa State       |                83 |       

In [14]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    max_tokens=1024,
    messages=[
        {"role": "user", "content": prompt}
    ]
)
Markdown(message.content[0].text)

Based on the 2008-09 Stanford Cardinal women's basketball team schedule, the top three highest-scoring games for Stanford were:

1. January 8, 2009 vs. Washington, where Stanford scored 112 points.
2. January 10, 2009 vs. Washington State, with Stanford putting up 102 points.
3. December 13, 2008 vs. Fresno State, in which Stanford scored 100 points.

In these three games, Stanford dominated their opponents, winning by large margins of 77, 49, and 38 points respectively.

In [16]:
from rouge import Rouge

ref = validate[12]['summary']
pred = message.content[0].text
rouge = Rouge()

rouge_results = rouge.get_scores(pred, ref)
print(rouge_results)

rouge_scores_out = []
for metric in ["rouge-1", "rouge-2", "rouge-l"]:
    for label in ["F-Score"]:
        eval_1_score = rouge_results[0][metric][label[0].lower()]

        row = {
            "Metric": f"{metric} ({label})",
            "Summary 1": eval_1_score
        }
        rouge_scores_out.append(row)

print(rouge_scores_out)

# Instantiate the BERTScorer object for English language
scorer = BERTScorer(lang="en")

# Calculate BERTScore for the summary 1 against the excerpt
# P1, R1, F1_1 represent Precision, Recall, and F1 Score respectively
P1, R1, F1_1 = scorer.score([pred], [ref])

print("Summary 1 F1 Score:", F1_1.tolist()[0])

[{'rouge-1': {'r': 0.6585365853658537, 'p': 0.46551724137931033, 'f': 0.5454545406019795}, 'rouge-2': {'r': 0.19298245614035087, 'p': 0.15492957746478872, 'f': 0.1718749950598146}, 'rouge-l': {'r': 0.5365853658536586, 'p': 0.3793103448275862, 'f': 0.4444444395918784}}]
[{'Metric': 'rouge-1 (F-Score)', 'Summary 1': 0.5454545406019795}, {'Metric': 'rouge-2 (F-Score)', 'Summary 1': 0.1718749950598146}, {'Metric': 'rouge-l (F-Score)', 'Summary 1': 0.4444444395918784}]


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Summary 1 F1 Score: 0.9067250490188599
